<a href="https://colab.research.google.com/github/emseekim/SSD-Object-Detection-Pet-Face/blob/main/07_3%ED%8C%80_%EA%B9%80%EB%AF%BC%EC%B2%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **1. 프로젝트 개요 (Project Overview)**

### **1-1. 문제 정의 및 목표**
SSD를 활용한 Oxford-IIIT Pet 얼굴 탐지 (Face Detection)
* **주제:** 반려동물(개, 고양이) 이미지에서 얼굴(Face) 영역을 식별하고 위치를 찾아내는 **Object Detection** 모델 개발.
* **배경:** 반려동물 시장이 성장함에 따라 동물의 상태를 모니터링하거나, 사진을 자동 분류하는 서비스에 대한 수요가 증가함. 이를 위해 이미지 내에서 동물의 위치와 종류를 정확히 파악하는 기술이 필요함.

### **1-2. 목표 및 평가지표**
1. 데이터 처리: XML 형식의 Annotation 파일을 파싱하여 이미지와 Bounding Box 정보를 매칭하는 전처리 파이프라인 구축.

2. 모델링: 속도와 정확도의 균형이 우수한 SSD (Single Shot Detector) 모델을 활용하여 Transfer Learning(전이 학습) 수행.

3. 학습 및 최적화: 학습 데이터에 맞춰 모델을 미세 조정(Fine-tuning)하고, 과적합을 방지하며 성능을 최적화.

4. 평가: **mAP(mean Average Precision)**와 IoU(Intersection over Union) 지표를 활용하여 모델의 탐지 성능을 정량적으로 평가하고, 시각화를 통해 정성적으로 확인.

### **1-3. 데이터 소개**
* **데이터셋:** [The Oxford-IIIT Pet Dataset](https://www.kaggle.com/datasets/devdgohil/the-oxfordiiit-pet-dataset)
    * `images`: 37종의 개와 고양이 이미지
    * `annotations`: 각 이미지 내 객체의 위치 정보(Bounding Box)를 담은 XML 파일

## **2. 환경 설정 및 데이터 준비**

### **2-1. 구글 드라이브 마운트**

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **2-1. 주요 라이브러리 임포트**

In [5]:
# 1. 표준 라이브러리
import os
import random

# 2. 서드파티 라이브러리
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2

# 3. 딥러닝 위한 핵심 도구
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision.models.detection import ssd300_vgg16
from torchvision.models.detection.ssd import SSD300_VGG16_Weights  # 미리 학습된 지식

# 4. XML 관련 도구 (핵심도구)
import xml.etree.ElementTree as ET

# 5. 좌표 변환 도구 (핵심도구)
import albumentations as A
from albumentations.pytorch import ToTensorV2

### **2-3. Seed 고정 및 Device 설정**

In [10]:
# 시드 고정
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print(f"모든 시드가 {seed}(으)로 고정되었습니다.")


# 2. 시드 고정 실행
SEED = 42
set_seed(SEED)

# 3. GPU 사용 가능 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 4. 설정된 장치 확인
print(f"현재 사용 가능한 장치(Device): {device}")

# 5. GPU 확인
if device.type == 'cuda':
    print(f"현재 할당된 GPU: {torch.cuda.get_device_name(0)}")

모든 시드가 42(으)로 고정되었습니다.
현재 사용 가능한 장치(Device): cuda
현재 할당된 GPU: Tesla T4


### **2-4. 데이터 경로 설정**

In [7]:
# zip 파일 경로
DRIVE_ZIP_PATH = '/content/drive/MyDrive/My_AI_Project/The_Oxfordiiit_Pet_Dataset.zip'

# 압축 해제 폴더 경로
LOCAL_DATA_PATH = '/content/data'

# Colab 로컬 환경으로 복사될 zip 파일의 이름 (경로에서 파일명만 추출)
local_zip_filename = os.path.basename(DRIVE_ZIP_PATH)

# colab 로컬 환경으로 복사될 zip 파일의 전체 경로
LOCAL_ZIP_PATH = f"/content/data/{local_zip_filename}"

print(f"드라이브 원본 경로: {DRIVE_ZIP_PATH}")
print(f"로컬 압축 해제 경로: {LOCAL_DATA_PATH}")
print(f"로컬 zip 파일 경로: {LOCAL_ZIP_PATH}")

드라이브 원본 경로: /content/drive/MyDrive/My_AI_Project/The_Oxfordiiit_Pet_Dataset.zip
로컬 압축 해제 경로: /content/data
로컬 zip 파일 경로: /content/data/The_Oxfordiiit_Pet_Dataset.zip


### **2-5. 데이터셋 복사 및 압축 해제**

In [27]:
# 데이터셋 복사 및 압축 해제

# os.path.exists(): 폴더 존재 여부 확인
# os.listdir(): 해당 경로 안의 파일/폴더 목록을 리스트로 변환 (비어있으면 빈 리스트 [])
# not.os.listdir(LOCAL_DATA_PATH): 폴더가 비어있으면 True

# 1. 로컬 데이터 폴더가 없거나 혹은 비어있으면 실행
if not os.path.exists(LOCAL_DATA_PATH) or not os.listdir(LOCAL_DATA_PATH):

    # (만약 비어있는 폴더가 이미 존재하는 경우를 대비해) 폴더 생성
    os.makedirs(LOCAL_DATA_PATH, exist_ok=True)

    if not os.listdir(LOCAL_DATA_PATH):  # 폴더가 비어있을 때만 복사/압축 진행
        print(f"'{LOCAL_DATA_PATH}' 폴더가 비어있어, 데이터를 새로 준비합니다.")

        # 드라이브에서 Colab 로컬로 zip 파일 복사
        print(f"드라이브에서 파일을 복사합니다...(목적지: {LOCAL_DATA_PATH}")
        !cp "{DRIVE_ZIP_PATH}" "{LOCAL_DATA_PATH}"

        # Colab 로컬에 압축 해제
        print(f"'{LOCAL_ZIP_PATH}' 파일의 압축을 '{LOCAL_DATA_PATH}'에 해제합니다...")
        !unzip -q "{LOCAL_ZIP_PATH}" -d "{LOCAL_DATA_PATH}"

        # (공간 확보) Colab에 복사했던 불필요한 zip 파일 삭제
        !rm "{LOCAL_ZIP_PATH}"
        print("압축 해제 완료! 데이터 준비 끝.")
    else:
        print(f"이미 데이터가 존재합니다. 건너뜁니다.")

else:
    # 폴더가 존재하고, 비어있지도 않으면 (이미 데이터가 있으면)
    print(f"폴더에 이미 데이터가 존재합니다. 준비를 건너뜁니다.")


폴더에 이미 데이터가 존재합니다. 준비를 건너뜁니다.


## **3. 데이터 전처리 및 파싱 (Preprocessing & Parsing)**

### **3-1. XML 파싱 함수 구현 (정답지 번역기)**
- 목적: XML 형식의 정답지(Annotation)를 파이썬과 모델이 이해할 수 있는 리스트 및 텐서(Tensor) 형태로 변환

- 주요 기능:
  - xml.etree.ElementTree 라이브러리를 통해 XML 파일을 읽음
  - 이미지 내의 모든 객체(object) 태그 탐색
  - 각 객체의 bndbox 좌표(xmin, ymin, xmax, ymax) 추출
  - 좌표와 라벨(Class)을 torch.Tensor 형식으로 변환하여 반환

In [28]:
# XML 파일을 읽어서 좌표(Box)와 정답(Lable)을 꺼내는 함수

# 경로 설정
xmls_dir = '/content/data/annotations/xmls'

def parse_xml(xml_path):
    # 1. XML 문서를 파싱해서 나무(Tree) 구조로 가져오기
    tree = ET.parse(xml_path)
    root = tree.getroot()  # 뿌리(Root) 태그 잡기

    boxes = []  # 박스 좌표를 담을 리스트
    labels = []  # 정답 라벨을 담을 리스트 (여기선 무조건 1=Face)

    # 2. <object> 태그를 하나씩 찾기 (사진 속에 물체가 여러 개일 수도 있으니까 for문 사용)
    for obj in root.findall('object'):
        # 3. <bndbox> 태그 찾기 (이 안에 좌표가 있음)
        bbox = obj.find('bndbox')

        # 4. 좌표 값 추출 (글자로 되어있으니 int로 바꿔줘야 함)
        xmin = int(bbox.find('xmin').text)
        ymin = int(bbox.find('ymin').text)
        xmax = int(bbox.find('xmax').text)
        ymax = int(bbox.find('ymax').text)

        # 5. 리스트에 담기
        # SSD는 좌표를 [xmin, ymin, xmax, ymax] 순서로 받음
        boxes.append([xmin, ymin, xmax, ymax])
        labels.append(1)  # 1: Face (개/고양이 얼굴)

    # 6. 텐서(Tensor)로 변환 (딥러닝은 텐서만 먹으니까)
    return torch.tensor(boxes, dtype=torch.float32), torch.tensor(labels, dtype=torch.int64)

# 파일이 있는지 확인
if os.path.exists(xmls_dir):
    file_list = os.listdir(xmls_dir)

    if len(file_list) > 0:
        # 가장 첫 번째 XML 파일을 가져와서 테스트
        sample_xml = os.path.join(xmls_dir, file_list(0))

        # 함수 실행!
        boxes, labels = parse_xml(sample_xml)

        print(f"테스트 파일: {sample_xml}")
        print(f"추출된 박스: \n{boxes}")
        print(f"추출된 라벨: \n{labels}")

    else:
        print(f"폴더는 있는데 파일이 없어: {xmls_dir}")

else:
    print(f"경로 오류: '{xmls_dir}' 폴더를 찾을 수 없어. 데이터 준비 단계를 다시 확인")

경로 오류: '/content/data/annotations/xmls' 폴더를 찾을 수 없어. 데이터 준비 단계를 다시 확인


In [29]:
import os

print("🕵️‍♂️ 구글 드라이브에서 .zip 파일을 찾는 중입니다...")

# 검색을 시작할 위치 (내 드라이브 최상단)
search_dir = '/content/drive/MyDrive'

found_files = []

# 드라이브를 돌면서 zip 파일 찾기 (너무 깊이 들어가면 오래 걸리니까 2단계 깊이까지만)
for root, dirs, files in os.walk(search_dir):
    level = root.replace(search_dir, '').count(os.sep)
    if level > 2: continue # 너무 깊은 폴더는 패스

    for file in files:
        if file.endswith('.zip') or file.endswith('.tar.gz'):
            full_path = os.path.join(root, file)
            print(f"📦 발견! -> {full_path}")
            found_files.append(full_path)

if not found_files:
    print("🚨 .zip 파일을 하나도 못 찾았어! 드라이브에 파일이 잘 올라갔는지 확인해봐.")

🕵️‍♂️ 구글 드라이브에서 .zip 파일을 찾는 중입니다...
📦 발견! -> /content/drive/MyDrive/KMSAuto Net 2015 v1.4.5 Portable.zip
📦 발견! -> /content/drive/MyDrive/Selected Images (4).zip
📦 발견! -> /content/drive/MyDrive/Selected Images (3).zip
📦 발견! -> /content/drive/MyDrive/Selected Images (2).zip
📦 발견! -> /content/drive/MyDrive/Selected Images (1).zip
📦 발견! -> /content/drive/MyDrive/Selected Images.zip
📦 발견! -> /content/drive/MyDrive/한컴오피스 2018.zip


KeyboardInterrupt: 